In [2]:
import os
import PyPDF2
import re
import psycopg2
import sys

def walk_dir(dir,fileinfo,topdown=True):
    for root, dirs, files in os.walk(dir, topdown):
        for name in files:
            fileinfo.write(os.path.join(root,name) + '\n')

def parse(directory):
    list=[]
    county = ""
    r = ""
    pdfFileObj = open(directory, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    num_pages = pdfReader.numPages
    for i in range(num_pages):
        pageObj = pdfReader.getPage(i)
        text = pageObj.extractText()

        text = text.replace("\n","")
        text = text.replace("Œ","-")
        text = text.replace("™","'")
        text = text.replace("˚","ffi")
        text = text.replace("˛","ff")
        text = text.replace("˜","fi")
        text = text.replace("˝","fl")

        content=text.split("Notice is hereby ")
        for j in range(len(content)):
            if j == 0:
                regions  = re.split("((?:[A-Z])(?:[A-Z|\.]+\s)+(?:REGION))", content[0])
            else:
                content=content[1].split("etc.)")
                regions  = re.split("((?:[A-Z])(?:[A-Z|\.]+\s)+(?:REGION))", content[1])
            
            for region in regions:

               if re.search('.+(REGION)',region):
                   r = str(region).strip()
               else:
                  area_ints = region.split("AREA: ")
                  for area_int in area_ints:
                      areas = area_int.split("& adjacent customers.")
                      long_street = "" 
                      for area in areas:

                         if re.search('COUNTY',area):
                            county = str(area).strip()
                         else:

                             if not re.search('^\s*$',area):
                                a=""

                                if re.search('.*(?=DATE)',area):
                                    area_break = re.search('.*(?=DATE)',area).end()
                                    a = area[:area_break].strip()

                                areas_=a.split(",")
                                for area_ in areas_:
                                    if re.search('PART OF|PARTS OF',area_):
                                        area__break=re.search('PART OF|PARTS OF',area_).end()
                                        area_=area_[area__break:].strip()
                                    list.append([r,area_.strip()])
    pdfFileObj.close()
    return list
                                    
dir = r'C:\GridWatch\Parsing service interruptions\Readable pdfs' #dir of folder
fileinfo = open('C:\\GridWatch\\Parsing service interruptions\\test2.txt','w')
walk_dir(dir,fileinfo)
fileinfo.close()

f = open('C:\\GridWatch\\Parsing service interruptions\\test2.txt','r')
#o = open('C:\\GridWatch\\Parsing service interruptions\\list1.txt','w')
master=[]
con = psycopg2.connect(database='outage_map', user='postgres') 
#con = psycopg2.connect(database='capstone', user='capstone', password='capstone', host='141.212.11.206', port='5432')
con.autocommit = True
cur = con.cursor()
for line in f:
    directory = line.replace("\n","")
    list = parse(directory)
    for area in list:
        if area not in master and len(area[1])>2:
            master.append(area)

cur.execute("DROP TABLE IF EXISTS area")
cur.execute("CREATE TABLE area(index SERIAL PRIMARY KEY)")
cur.execute("ALTER TABLE area ADD region text")
cur.execute("ALTER TABLE area ADD area text")
for element in master:
    for j in range(2):
        if element[j]=="":
            element[j]=None
    #print(element)

try:
    dataText = ', '.join(map(bytes.decode,(cur.mogrify('(%s,%s)', element) for element in master)))
    cur.execute('INSERT INTO area (region, area) VALUES ' + dataText)
    #cur.execute("INSERT INTO area (region, area) VALUES (%s, %s)",element)
except Exception as e:
    print(e)
    print(directory)
    print(list)
    print("----------------------------")
f.close()
#o.close()
con.commit()
print("finish")

finish


In [9]:
import psycopg2
import sys

#con = psycopg2.connect(database='outage_map', user='postgres')
con = psycopg2.connect(database='capstone', user='capstone', password='capstone', host='141.212.11.206', port='5432')
con.autocommit = True
cur = con.cursor()
cur.execute("SELECT MAX(index) FROM area")
count = cur.fetchall()
print(count[0])

(8331,)
